In [23]:
# !pip install pandas
# !pip install matplotlib
# !pip install numpy
# !pip install seaborn
# !pip install requests
# !pip install urllib3

In [19]:
# !pip install -r requirements.txt

ERROR: Could not find a version that satisfies the requirement matplotlib>=3.4.3 (from versions: 0.86, 0.86.1, 0.86.2, 0.91.0, 0.91.1, 1.0.1, 1.1.0, 1.1.1, 1.2.0, 1.2.1, 1.3.0, 1.3.1, 1.4.0, 1.4.1rc1, 1.4.1, 1.4.2, 1.4.3, 1.5.0, 1.5.1, 1.5.2, 1.5.3, 2.0.0b1, 2.0.0b2, 2.0.0b3, 2.0.0b4, 2.0.0rc1, 2.0.0rc2, 2.0.0, 2.0.1, 2.0.2, 2.1.0rc1, 2.1.0, 2.1.1, 2.1.2, 2.2.0rc1, 2.2.0, 2.2.2, 2.2.3, 2.2.4, 2.2.5, 3.0.0rc2, 3.0.0, 3.0.1, 3.0.2, 3.0.3, 3.1.0rc1, 3.1.0rc2, 3.1.0, 3.1.1, 3.1.2, 3.1.3, 3.2.0rc1, 3.2.0rc3, 3.2.0, 3.2.1, 3.2.2, 3.3.0rc1, 3.3.0, 3.3.1, 3.3.2, 3.3.3, 3.3.4)
ERROR: No matching distribution found for matplotlib>=3.4.3


In [1]:
import platform
print(platform.python_version())

3.6.13


In [13]:
import json
import pandas as pd
import os
import shutil
from datetime import datetime

DATA_DIR = "../data"
DATA_FILE = os.path.join(DATA_DIR, "data.json")
DUMP_FILE = os.path.join(DATA_DIR, "dump.pkl")

In [14]:
store_columns = (
    "id",  # 음식점 고유번호
    "store_name",  # 음식점 이름
    "branch",  # 음식점 지점 여부
    "area",  # 음식점 위치
    "tel",  # 음식점 번호
    "address",  # 음식점 주소
    "latitude",  # 음식점 위도
    "longitude",  # 음식점 경도
    "category",  # 음식점 카테고리
    "review_cnt", # 평가 수
)

review_columns = (
    "id",  # 리뷰 고유번호
    "store",  # 음식점 고유번호
    "user",  # 유저 고유번호
    "score",  # 평점
    "content",  # 리뷰 내용
    "reg_time",  # 리뷰 등록 시간
)

menu_columns = (
    "id", # 메뉴 고유번호
    "store", # 음식점 고유번호
    "menu_name", # 메뉴 이름
    "price", #메뉴 가격
)

user_columns = (
    "id", #유저 고유번호
    "gender", #유저 성별
    "age", #유저 나이
)

In [15]:
def import_data(data_path=DATA_FILE):
    """
    Req. 1-1-1 음식점 데이터 파일을 읽어서 Pandas DataFrame 형태로 저장합니다
    """

    try:
        with open(data_path, encoding="utf-8") as f:
            data = json.loads(f.read())
    except FileNotFoundError as e:
        print(f"`{data_path}` 가 존재하지 않습니다.")
        exit(1)

    stores = []  # 음식점 테이블
    reviews = []  # 리뷰 테이블
    menus = [] # 메뉴 테이블
    users = [] # 유저 테이블
 
    for d in data:

        categories = [c["category"] for c in d["category_list"]]
        stores.append(
            [
                d["id"],
                d["name"],
                d["branch"],
                d["area"],
                d["tel"],
                d["address"],
                d["latitude"],
                d["longitude"],
                "|".join(categories), # 구분자 | 를 이용하여 합침
                d["review_cnt"], # 리뷰 갯수
            ]
        )
    
        for review in d["review_list"]:
            r = review["review_info"]
            u = review["writer_info"]

            reviews.append(
                [r["id"], d["id"], u["id"], r["score"], r["content"], r["reg_time"]]
            )
            
            users.append(
                [u["id"], u["gender"], datetime.today().year - int(u["born_year"])+1]
            )

        for menu in d["menu_list"]:
           
            menus.append(
                [len(menus) + 1, d["id"], menu["menu"], menu["price"],]
            )
        
    store_frame = pd.DataFrame(data=stores, columns=store_columns)
    review_frame = pd.DataFrame(data=reviews, columns=review_columns)
    menu_frame = pd.DataFrame(data=menus, columns=menu_columns)
    user_frame = pd.DataFrame(data=users, columns=user_columns)
    user_frame.sort_values(by=['id'], axis=0)
    
    return {"stores": store_frame, "reviews": review_frame, "menus": menu_frame, "users": user_frame}

In [16]:
def dump_dataframes(dataframes):
    pd.to_pickle(dataframes, DUMP_FILE)


def load_dataframes():
    return pd.read_pickle(DUMP_FILE)


def main():

    print("[*] Parsing data...")
    data = import_data()
    print("[+] Done")

    print("[*] Dumping data...")
    dump_dataframes(data)
    print("[+] Done\n")

    data = load_dataframes()

    term_w = shutil.get_terminal_size()[0] - 1
    separater = "-" * term_w

    print("[음식점]")
    print(f"{separater}\n")
    print(data["stores"].head())
    print(f"\n{separater}\n\n")

    print("[리뷰]")
    print(f"{separater}\n")
    print(data["reviews"].head())
    print(f"\n{separater}\n\n")

    print("[메뉴]")
    print(f"{separater}\n")
    print(data["menus"].head())
    print(f"\n{separater}\n\n")
    
    print("[유저]")
    print(f"{separater}\n")
    print(data["users"].head())
    print(f"\n{separater}\n\n")


if __name__ == "__main__":
    main()


[*] Parsing data...
[+] Done
[*] Dumping data...
[+] Done

[음식점]
-------------------------------------------------------------------------------

   id     store_name branch  area            tel                address  \
0   1           Agal   None    홍대  010-6689-5886   서울특별시 마포구 동교동 170-13   
1   2         Assisy   None    광주   062-367-0700    광주광역시 서구 농성동 631-33   
2   3  Battered Sole   None   이태원    02-749-6867   서울특별시 용산구 이태원동 118-9   
3   4      Chakyoung   None  달맞이길   051-756-5566  부산광역시 해운대구 중2동 1509-5   
4   5       Delabobo   None   발산역   02-2667-9854      서울특별시 강서구 등촌동 689   

    latitude   longitude   category  review_cnt  
0  37.556862  126.926666   아구찜|포장마차           0  
1  35.150746  126.890062         카페           0  
2  37.535032  126.991664    피쉬앤칩스|펍           0  
3  35.158587  129.175004  레스토랑|카프레제           0  
4  37.559904  126.840512  디저트카페|디저트           0  

-------------------------------------------------------------------------------


[리뷰]
---------------

In [6]:
print(type(datetime.today().year))


<class 'int'>


Analyze.py 
1. Req. 1-2-1 각 음식점의 평균 평점을 계산하여 높은 평점의 음식점 순으로 `n`개의 음식점을 정렬하여 리턴합니다
2. Req. 1-2-2 리뷰 개수가 `min_reviews` 미만인 음식점은 제외합니다.
3. Req. 1-2-3 가장 많은 리뷰를 받은 `n`개의 음식점을 정렬하여 리턴합니다
4. Req. 1-2-4 가장 많은 리뷰를 작성한 `n`명의 유저를 정렬하여 리턴합니다.

In [7]:
from parse import load_dataframes
import pandas as pd
import shutil

In [8]:
def sort_stores_by_score(dataframes, n=20, min_reviews=30):
    """
    Req. 1-2-1 각 음식점의 평균 평점을 계산하여 높은 평점의 음식점 순으로 `n`개의 음식점을 정렬하여 리턴합니다
    Req. 1-2-2 리뷰 개수가 `min_reviews` 미만인 음식점은 제외합니다.
    """
    stores_reviews = pd.merge(
        dataframes["stores"], dataframes["reviews"], left_on="id", right_on="store"
    )
    # 리뷰 갯수 min_reviews 이상인 음식점
    scores_group = stores_reviews[stores_reviews["review_cnt"] >= min_reviews].groupby(["store", "store_name", "branch", "review_cnt"])
    # 각 음식점의 평균 평점 중 높은 평점의 음식점 순 정렬
    scores = scores_group.mean().sort_values(by='score', ascending=False)
    return scores.head(n=n).reset_index()

In [9]:
def get_most_reviewed_stores(dataframes, n=20):
    """
    Req. 1-2-3 가장 많은 리뷰를 받은 `n`개의 음식점을 정렬하여 리턴합니다
    """
    # review 순으로 store를 정렬하기 위해
    # store와 review 데이터 프레임을 병합한다.
    stores_reviews = pd.merge(
        dataframes["stores"], dataframes["reviews"], left_on="id", right_on="store"
    )
    # 모든 컬럼을 다 확인하려고, 모든 컬럼을 넣음
    scores_group = stores_reviews.groupby(
        [
            "store",
            "store_name",
            "branch",
            "area",
            "tel",
            "address",
            "latitude",
            "longitude",
            "category",
            "review_cnt", # 평가 수
        ]
    )
    # 리뷰 순으로 정렬
    scores_group_sorted_by_rc = (
        scores_group.store.count()
        .reset_index(name="review_count")
        .sort_values(["review_count"], ascending=False)
    )
    # 상위 n개의 음식점을 반환
    return scores_group_sorted_by_rc.head(n=n).reset_index()


In [236]:
def get_most_active_users(dataframes, n=20):
    """
    Req. 1-2-4 가장 많은 리뷰를 작성한 `n`명의 유저를 정렬하여 리턴합니다.
    """

    users_reviews = pd.merge(
        dataframes["users"], dataframes["reviews"], left_on="id", right_on="user"
    )
    users = users_reviews.groupby(["user", "gender", "age"]).size().reset_index().rename(columns={0: 'reviews_cnt'})

    users_sorted_by_review_counts = users.sort_values(by='reviews_cnt', ascending=False)
    return users_sorted_by_review_counts

In [238]:
def main():
    data = load_dataframes()

    term_w = shutil.get_terminal_size()[0] - 1
    separater = "-" * term_w

    stores_most_scored = sort_stores_by_score(data)
    reviews_most_scored = get_most_reviewed_stores(data)
    users_most_scored = get_most_active_users(data)


    print("[최고 평점 음식점]")
    print(f"{separater}\n")
    for i, store in stores_most_scored.iterrows():
        print(
            "{rank}위: {store}({score}점)".format(
                rank=i + 1, store=store.store_name, score=store.score
            )
        )
    print(f"\n{separater}\n\n")

    print("[리뷰가 많은 음식점]")
    print(f"{separater}\n")
    for i, store in reviews_most_scored.iterrows():
        print(
            "{rank}위 : {store}({review_count}개)".format(
                rank=i + 1, store=store.store_name, review_count=store.review_cnt
            )
        )
    print(f"\n{separater}\n\n")

    print("[리뷰를 많이 작성한 유저]")
    print(f"{separater}\n")
    for i, store in users_most_scored.iterrows():
        print(
            "{rank}위 : {store}({review_count}개)".format(
                rank=i + 1, store=store.store_name, review_count=store.review_cnt
            )
        )
    print(f"\n{separater}\n\n")


if __name__ == "__main__":
    main()

[최고 평점 음식점]
-------------------------------------------------------------------------------



AttributeError: 'Series' object has no attribute 'score'

visualize.py

In [3]:
pip install folium

  Using cached folium-0.12.1-py2.py3-none-any.whl (94 kB)
  Using cached branca-0.4.2-py3-none-any.whl (24 kB)
Note: you may need to restart the kernel to use updated packages.


In [17]:
import itertools
from collections import Counter
from os import name
from parse import load_dataframes
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
import folium

In [18]:
def set_config():
    # 폰트, 그래프 색상 설정
    font_list = fm.findSystemFonts(fontpaths=None, fontext="ttf")
    if any(["notosanscjk" in font.lower() for font in font_list]):
        plt.rcParams["font.family"] = "Noto Sans CJK JP"
    else:
        if not any(["malgun" in font.lower() for font in font_list]):
            raise Exception(
                "Font missing, please install Noto Sans CJK or Malgun Gothic. If you're using ubuntu, try `sudo apt install fonts-noto-cjk`"
            )

        plt.rcParams["font.family"] = "Malgun Gothic"

    sns.set_palette(sns.color_palette("Spectral"))
    plt.rc("xtick", labelsize=6)

In [19]:
def show_store_categories_graph(dataframes, n=100):
    """
    Tutorial: 전체 음식점의 상위 `n`개 카테고리 분포를 그래프로 나타냅니다.
    """
    stores = dataframes["stores"]
    # 모든 카테고리를 1차원 리스트에 저장합니다
    categories = stores.category.apply(lambda c: c.split("|"))
    categories = itertools.chain.from_iterable(categories)

    # 카테고리가 없는 경우 / 상위 카테고리를 추출합니다
    categories = filter(lambda c: c != "", categories)
    categories_count = Counter(list(categories))
    best_categories = categories_count.most_common(n=n) # 카테고리가 없는 것 중 가장 동일한 자료가 많은 것을 구함.
    df = pd.DataFrame(best_categories, columns=["category", "count"]).sort_values(
        by=["count"], ascending=False
    )

    # 그래프로 나타냅니다
    chart = sns.barplot(x="category", y="count", data=df)
    chart.set_xticklabels(chart.get_xticklabels(), rotation=45)
    plt.title("음식점 카테고리 분포")
    plt.show()

In [20]:
def show_store_review_distribution_graph(dataframes):
    """
    Req. 1-3-1 전체 음식점의 리뷰 개수 분포를 그래프로 나타냅니다. 
    """
    stores = dataframes["stores"]
    # 리뷰 갯수를 1차원 리스트에 저장합니다.
    reviews_cnt = (
        stores.groupby("review_cnt").review_cnt.count().reset_index(name="count")
    )
    # print(reviews_cnt)
    chart = sns.barplot(x="review_cnt", y="count", data=reviews_cnt)
    # matplotlib.pylplot.yscale
    # value : linear, log, symlog, logit
    chart.set(yscale="log")
    plt.title("전체 음식점의 리뷰 개수 분포")
    plt.show()


In [21]:
def show_store_average_ratings_graph(dataframes):
    """
    Req. 1-3-2 각 음식점의 평균 평점을 그래프로 나타냅니다.
    """
    # 음식점과 리뷰 data merge
    stores_reviews = pd.merge(
        dataframes["stores"], dataframes["reviews"], left_on="id", right_on="store"
    )
    #store로 grouping 후 해당 store의 score(평점)의 평균을 mean_score colomn으로 만들고 reset_index
    store_mean_score = (
        stores_reviews.groupby("store").score.mean().reset_index(name="mean_score")
    )
    # store_maen_score에서 mean_score의 첫째 자리수에서 반올림(decimals=1)
    store_mean_score["mean_score"] = store_mean_score["mean_score"].round(decimals=1)
    # 리뷰 평점의 store의갯수를 count로 reset_index 
    mean_score_count = (
        store_mean_score.groupby("mean_score").store.count().reset_index(name="count")
    )
    chart = sns.barplot(x="mean_score", y="count", data=mean_score_count)
    chart.set(yscale="log")
    plt.title("각 음식점의 평균 평점")
    plt.show()

In [22]:
def show_user_review_distribution_graph(dataframes):
    """
    Req. 1-3-3 전체 유저의 리뷰 개수 분포를 그래프로 나타냅니다.
    """
    # 유저와 리뷰 
    users_reviews = pd.merge(
        dataframes["users"], dataframes["reviews"], left_on="id", right_on="user"
    )
    users_review_cnt = (
        users_reviews.groupby(["user"]).user.count().reset_index(name="review_cnt")
    )
    grouped = (
        users_review_cnt.groupby(["review_cnt"]).review_cnt.count().reset_index(name="count")
    )
    chart = sns.barplot(x="review_cnt", y="count", data=grouped)
    chart.set(yscale="log")
    plt.title("전체 유저의 리뷰 개수")
    plt.show()

In [23]:
def show_user_age_gender_distribution_graph(dataframes):
    """
    Req. 1-3-4 전체 유저의 성별/나이대 분포를 그래프로 나타냅니다.
    """
    users_reviews = pd.merge(
        dataframes["users"], dataframes["reviews"], left_on="id", right_on="user"
    )
    users_age_gender = (
        users_reviews.groupby(["gender", "user", "age"]).user.count().reset_index(name="user_age_gender_cnt")
    )
    # 범위를 벗어난 나이값 제거
    users_age_gender = users_age_gender[users_age_gender["age"] > 0]
    users_age_gender = users_age_gender[users_age_gender["age"] < 100] 
    
    # print(users_age_gender.head())
    chart = sns.countplot(x="age", hue="gender", data=users_age_gender)
    chart.set_yticklabels(chart.get_yticklabels(), rotation=100)
    plt.title("전체 유저의 성별/나이대")
    plt.show()

In [32]:
def show_stores_distribution_graph(dataframes):
    """
    Req. 1-3-5 각 음식점의 위치 분포를 지도에 나타냅니다.
    """
    # Folium 라이브러리
    # 지역, 리뷰수, 평점을 주어 나타내라
    stores_reviews = pd.merge(
        dataframes["stores"], dataframes["reviews"] , left_on="id", right_on="store"
    )
    store_place = stores_reviews.groupby(["store", "store_name", "review_cnt", "latitude", "longitude"])["score"].mean().reset_index(name="mean_score")
    store_place = store_place.astype({"latitude": float, "longitude": float})
    store_place["mean_score"] = store_place["mean_score"].round(decimals=1)
    store_place = (
        store_place[(store_place["review_cnt"] >= 10) & (store_place["mean_score"] >=4)].reset_index().drop(columns=["index"], axis=1)
    )   
    lat = store_place["latitude"].mean()
    lng = store_place["longitude"].mean()
    

    m = folium.Map([lat, lng], zoom_start=10)
    for i in store_place.index:
        n_lat = store_place.loc[i, "latitude"]
        n_lng = store_place.loc[i, "longitude"]
        n_name = store_place.loc[i, "store_name"]
        folium.Marker([n_lat, n_lng], tooltip = n_name).add_to(m)
    m

In [33]:
def main():
    set_config()
    data = load_dataframes()
    # show_store_categories_graph(data)
    # show_store_review_distribution_graph(data)
    # show_store_average_ratings_graph(data)
    # show_user_review_distribution_graph(data)
#     show_user_age_gender_distribution_graph(data)
    show_stores_distribution_graph(data)

if __name__ == "__main__":
    main()